In [1744]:
import plotly.express as px
import pandas as pd
import math

In [1745]:
m_b = 369337151152699193054055607757259347374982104504410195275125903895224556366311251446376608239822856138494863695134673125683287947156143694233250334267089102735426277711773226904880345676858215825651777505248888532875008085500643909686874148851636151483956370057721387871124395600324347768383994380355036272409112974299620569402348404736
m_s = 114148972250068685860540806384444273403507773433731256927856097780659346614538639718747307860228582881080413030584975590967077830044685954704962110942689761870469677315927496013162717063644241469591274756759902873371054704228142684457547167376599754212227669828788080215445802833479098419433248217628632384036280734120751793761548900625
min_delta_b = 6646
min_delta_s = 1
max_delta_b = 7620
max_delta_s = 22355
stepping = "buying"
slopes = [[
        (6646, 21504, 0.3924056761549752),
        (6647, 21507, 0.15683426563378278),
# new optimum; distance: 1n stepsize: 1n
        (6648, 21511, 0.9212628551125903),
# worsened; distance: 0n stepsize: 1n
# worsened; distance: 0n stepsize: 1n
        (6649, 21514, 0.6856914445913977),
# extrapolated:
        (6651, 21520, 0.21454862354901294),
        (6652, 21524, 0.9789772130278205),
# worsened; distance: 4n stepsize: 1n
# new stepsize: 4n
# worsened; distance: 4n stepsize: 1n
        (6653, 21527, 0.743405802506628),
        (6656, 21536, 0.036691570943050734),
# new optimum; distance: 9n stepsize: 4n
# new stepsize: 9n
        (6657, 21540, 0.8011201604218582),
        (6660, 21549, 0.09440592885828093),
# worsened; distance: 0n stepsize: 4n
# worsened; distance: 0n stepsize: 4n
        (6661, 21553, 0.8588345183370883),
        (6664, 21562, 0.15212028677351114),
# worsened; distance: 4n stepsize: 4n
# worsened; distance: 4n stepsize: 4n
        (6665, 21566, 0.9165488762523186),
        (6668, 21575, 0.20983464468874133),
        (6673, 21591, 0.031977592082779074),
# new optimum; distance: 17n stepsize: 9n
# new stepsize: 17n
# extrapolated:
        (6775, 21921, 0.0036937189211491473),
        (6776, 21925, 0.7681223083999567),
        (6779, 21934, 0.06140807683637934),
# worsened; distance: 0n stepsize: 4n
# worsened; distance: 0n stepsize: 4n
        (6780, 21938, 0.8258366663151868),
        (6783, 21947, 0.11912243475160954),
        (6796, 21989, 0.05669409797610769),
# extrapolated:
        (6909, 22355, 0.43712470908136075),
]]

In [1746]:
slopes_ = []
for slope in slopes:
    slope_ = []
    slope__ = []
    prevloss = None
    for (delta_b, delta_s, _) in slope:
        # if stepping == "buying":
        #     loss = (-m_b * buying % m_s) / m_s
        # else:
        #     loss = (m_s * selling % m_b) / m_b
        loss = delta_s * m_s / m_b - math.floor(delta_s * m_s / m_b)
        slope__.append((delta_b, delta_s, loss))
        if prevloss is not None and prevloss < loss:
            prevloss = loss
            loss = -loss
        else:
            prevloss = loss

        slope_.append((delta_b, delta_s, loss))
    slopes_.append(slope_)
    slopes_.append(slope__)
slopes = slopes_

In [1747]:
perfect = []
best_imperfect = []
other_imperfect = []
delta_b_too_low = []
suboptimal_delta_b = []
prev_delta_b = None
best_imperfect_loss = None
for delta_s in range(min_delta_s, max_delta_s + 1):
    delta_b = math.floor(delta_s * m_s / m_b)
    # if (delta_b == 0):
    #     print("delta_b == 0 for", delta_s)
    #     continue
    loss = delta_s * m_s / m_b - math.floor(delta_s * m_s / m_b)
    if (delta_b > max_delta_b):
        delta_b = max_delta_b
    if (delta_b < min_delta_b):
        delta_b_too_low.append((delta_b, delta_s, loss))
        continue
    if (delta_b == prev_delta_b):
        suboptimal_delta_b.append((delta_b, delta_s, loss))
        continue
    prev_delta_b = delta_b
    if delta_b * m_b == delta_s * m_s:
      perfect.append((delta_b, delta_s, loss))
    elif best_imperfect_loss is None or loss <= best_imperfect_loss:
      best_imperfect_loss = loss
      best_imperfect.append((delta_b, delta_s, loss))
    else:
      other_imperfect.append((delta_b, delta_s, loss))
    if (delta_b == max_delta_b):
        break
    
delta_b_too_low = []

In [1748]:
# Convert your data lists to Pandas DataFrames for easier handling
df1 = pd.DataFrame(perfect, columns=['delta_b', 'delta_s', 'loss'])
df2 = pd.DataFrame(best_imperfect, columns=['delta_b', 'delta_s', 'loss'])
df3 = pd.DataFrame(other_imperfect, columns=['delta_b', 'delta_s', 'loss'])
df4 = pd.DataFrame(delta_b_too_low, columns=['delta_b', 'delta_s', 'loss'])
df5 = pd.DataFrame(suboptimal_delta_b, columns=['delta_b', 'delta_s', 'loss'])
# df6 = pd.DataFrame(first_slope, columns=['delta_b', 'delta_s', 'loss'])
# df7 = pd.DataFrame(second_slope, columns=['delta_b', 'delta_s', 'loss'])
slopes_df = [pd.DataFrame(slope, columns=['delta_b', 'delta_s', 'loss']) for slope in slopes]
for i in range(len(slopes_df)):
    slopes_df[i]['Series'] = 'slope' + str(i + 1)

# Combine the dataframes with an identifier column
df1['Series'] = 'Without'
df2['Series'] = 'best'
df3['Series'] = 'other'
df4['Series'] = 'too low'
df5['Series'] = 'subopt'
# df6['Series'] = '1st'
# df7['Series'] = '2nd'
# pd.concat([df5, df1, df3] + slopes_df + [df2])
combined_df = pd.concat([df4, df5, df1, df3, df2]) 

# # Add a new column for point size based on the series
# # Assuming 'Series' is a column that identifies each series
# series_size_mapping = {series: i+1 for i, series in enumerate(combined_df['Series'].unique())}
# combined_df['point_size'] = combined_df['Series'].map(series_size_mapping)

# # Sort the DataFrame in reverse order of point size
# combined_df = combined_df.sort_values(by='point_size', ascending=False)

In [1749]:

# Create an interactive scatter plot
fig = px.scatter(combined_df, x='delta_b', y='loss', color='Series', #size='point_size',
                 hover_data=['delta_b', 'delta_s', 'loss'])

# Add lines for specified series
# for series in slopes_df:
#     fig.add_trace(px.line(series, x='delta_s', y='loss', color='Series').data[0])

# Concatenate all DataFrames in slopes_df
all_slopes_df = pd.concat(slopes_df)
# Create a line plot for all slopes
line_fig = px.line(all_slopes_df, x='delta_b', y='loss', color='Series', hover_data=['delta_b', 'delta_s', 'loss'], markers=True)

# Add each line trace individually
for trace in line_fig.data:
    trace.update(marker=dict(symbol='circle-open', size=10)) 
    fig.add_trace(trace)

# Update marker properties to have darker outlines
# fig.update_traces(marker=dict(line=dict(width=1, color='rgba(40,40,40,1)')))  # Adjust the color as needed

# Update marker properties to remove outlines or match the fill color
fig.update_traces(marker=dict(line=dict(width=0)))  # Removes the outline

# Alternatively, to set the outline to the same color as the fill:
# fig.update_traces(marker=dict(line=dict(width=1, color='SAME_COLOR')))


# Update the layout for dark mode
fig.update_layout(
    title='Scatter Plot in Dark Mode',
    plot_bgcolor='rgb(17,17,17)',  # Dark plot background
    paper_bgcolor='rgb(17,17,17)',  # Dark paper background
    font_color='white',  # White text for better contrast
    xaxis=dict(
        gridcolor='rgb(80,80,80)',  # Darker gridlines
    ),
    yaxis=dict(
        gridcolor='rgb(80,80,80)',  # Darker gridlines
    )
)

# Add plot titles and labels
fig.update_layout(title='effective loss by delta buying',
                  xaxis_title='delta b',
                  yaxis_title='loss',
                  # showlegend=False,
                  height=1600)
                  

# fig.update_yaxes(range=[combined_df['loss'].min(), combined_df['loss'].max()])  # Adjust the range as needed

# # Sort the DataFrame by 'ratio' in descending order
# sorted_df = combined_df.sort_values(by='loss', ascending=False)

# # Adding a line that connects the dots in decreasing order of their y-value
# fig.add_trace(px.line(sorted_df, x='delta_b', y='loss').data[0])

# # Find the five lowest unique y-values (ratios)
# lowest_ratios = combined_df['loss'].unique()
# lowest_ratios.sort()

# # Group by 'ratio' and draw lines for each of the five lowest ratios
# for loss in lowest_ratios[:20]:
#     group = combined_df[combined_df['loss'] == loss]
#     # Sort each group by 'delta_b' to make line connections logical
#     sorted_group = group.sort_values(by='delta_b')
#     fig.add_trace(px.line(sorted_group, x='delta_b', y='loss').data[0])

# # Adjusting the plot size and layout
# fig.update_layout(title='Scatter Plot with Lines for Lowest Y-values',
#                   xaxis_title='Delta_b',
#                   yaxis_title='loss',
#                   height=800)

# Show the plot
fig.show()


In [1750]:
m_b / m_s;

3.2355714105211923

In [1751]:
m_s / m_b;

0.3090644195792662

In [1752]:
# # Create an interactive scatter plot
# fig = px.scatter(combined_df, x='delta_s', y='loss', color='Series', #size='point_size',
#                  hover_data=['delta_b', 'delta_s', 'loss'])

# # Add lines for specified series
# # for series in slopes_df:
# #     fig.add_trace(px.line(series, x='delta_s', y='loss', color='Series').data[0])

# # Concatenate all DataFrames in slopes_df
# all_slopes_df = pd.concat(slopes_df)
# # Create a line plot for all slopes
# line_fig = px.line(all_slopes_df, x='delta_s', y='loss', color='Series', hover_data=['delta_b', 'delta_s', 'loss'], markers=True)

# # Add each line trace individually
# for trace in line_fig.data:
#     trace.update(marker=dict(symbol='circle-open', size=10)) 
#     fig.add_trace(trace)

# # Update marker properties to have darker outlines
# # fig.update_traces(marker=dict(line=dict(width=1, color='rgba(40,40,40,1)')))  # Adjust the color as needed

# # Update marker properties to remove outlines or match the fill color
# fig.update_traces(marker=dict(line=dict(width=0)))  # Removes the outline

# # Alternatively, to set the outline to the same color as the fill:
# # fig.update_traces(marker=dict(line=dict(width=1, color='SAME_COLOR')))


# # Update the layout for dark mode
# fig.update_layout(
#     title='Scatter Plot in Dark Mode',
#     plot_bgcolor='rgb(17,17,17)',  # Dark plot background
#     paper_bgcolor='rgb(17,17,17)',  # Dark paper background
#     font_color='white',  # White text for better contrast
#     xaxis=dict(
#         gridcolor='rgb(80,80,80)',  # Darker gridlines
#     ),
#     yaxis=dict(
#         gridcolor='rgb(80,80,80)',  # Darker gridlines
#     )
# )

# # Add plot titles and labels
# fig.update_layout(title='effective loss by delta selling',
#                   xaxis_title='delta s',
#                   yaxis_title='loss',
#                   # showlegend=False,
#                   height=1600)
                  

# # fig.update_yaxes(range=[combined_df['loss'].min(), combined_df['loss'].max()])  # Adjust the range as needed

# # # Sort the DataFrame by 'ratio' in descending order
# # sorted_df = combined_df.sort_values(by='loss', ascending=False)

# # # Adding a line that connects the dots in decreasing order of their y-value
# # fig.add_trace(px.line(sorted_df, x='delta_b', y='loss').data[0])

# # # Find the five lowest unique y-values (ratios)
# # lowest_ratios = combined_df['loss'].unique()
# # lowest_ratios.sort()

# # # Group by 'ratio' and draw lines for each of the five lowest ratios
# # for loss in lowest_ratios[:20]:
# #     group = combined_df[combined_df['loss'] == loss]
# #     # Sort each group by 'delta_b' to make line connections logical
# #     sorted_group = group.sort_values(by='delta_b')
# #     fig.add_trace(px.line(sorted_group, x='delta_b', y='loss').data[0])

# # # Adjusting the plot size and layout
# # fig.update_layout(title='Scatter Plot with Lines for Lowest Y-values',
# #                   xaxis_title='Delta_b',
# #                   yaxis_title='loss',
# #                   height=800)

# # Show the plot
# fig.show()


In [1753]:
def prime_factors(n):
    i = 2
    factors = {}
    while i * i <= n:
        if n % i:
            i += 1
        else:
            n //= i
            factors[i] = factors.get(i, 0) + 1
    if n > 1:
        factors[n] = factors.get(n, 0) + 1
    return factors

In [1754]:
prime_factors(m_b);

{2: 728, 3: 244}

In [1755]:
prime_factors(m_s);

{5: 4, 23: 244}

In [1756]:
351 - 68;

283

In [1757]:
from sympy import symbols, solve, Eq

In [1758]:
x, m, b, x1, x2, n1, n2, d1, d2 = symbols("x m b x1 x2 n1 n2 d1 d2");
y1 = n1 / d1;
y2 = n2 / d2;
m = (y2 - y1) / (x2 - x1);
m = m.simplify();
m;

(-d1*n2 + d2*n1)/(d1*d2*(x1 - x2))

In [1759]:
b = y1 - m * x1;
b = b.simplify();
b;

(d1*n2*x1 - d2*n1*x2)/(d1*d2*(x1 - x2))

In [1760]:
f = m * x + b;
#f = f.simplify();
f;

x*(-d1*n2 + d2*n1)/(d1*d2*(x1 - x2)) + (d1*n2*x1 - d2*n1*x2)/(d1*d2*(x1 - x2))

In [1761]:
solve(Eq(f, 0), x)[0].simplify();

(d1*n2*x1 - d2*n1*x2)/(d1*n2 - d2*n1)